# A.4b - RGB + Synthetic Depth (4-Channel)

**Experiment**: A.4b
**Input**: RGB (3-ch) + Synthetic Depth (1-ch) = 4-channel fusion
**Objective**: Test if RGB+Synthetic Depth can match RGB+Real Depth performance
**Comparison**: 
- A.1 RGB: mAP50=0.873, mAP50-95=0.370
- A.3 Real RGBD: mAP50=0.869, mAP50-95=0.379

**Hypothesis**: mAP50-95 slightly < A.3 but > A.1 (if successful, eliminates need for depth sensor)

**Prerequisites**:
1. Synthetic depth generated (from `generate_synthetic_depth.ipynb`)
2. Data prepared (from `prepare_synthetic_depth_data.py`)

---

In [ ]:
# Setup
import os
import torch
import numpy as np
from pathlib import Path
from ultralytics import YOLO
import pandas as pd
from datetime import datetime

IS_KAGGLE = os.path.exists('/kaggle/input')
BASE_PATH = Path('/kaggle/working' if IS_KAGGLE else r'd:\Work\Assisten Dosen\Anylabel\Experiments')
RGB_PATH = Path('/kaggle/input/ffb-localization' if IS_KAGGLE else BASE_PATH / 'datasets' / 'ffb_localization')
DEPTH_PATH = Path('/kaggle/input/ffb-synthetic-depth' if IS_KAGGLE else BASE_PATH / 'datasets' / 'depth_synthetic_da2')

print(f"Environment: {'Kaggle' if IS_KAGGLE else 'Local'}")
print(f"RGB: {RGB_PATH}")
print(f"Depth: {DEPTH_PATH}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
# Create YAML config
config_path = BASE_PATH / 'configs' / 'ffb_localization_rgbd_synthetic.yaml'
config_path.parent.mkdir(exist_ok=True)

yaml_content = f"""# RGBD Synthetic Dataset
path: {RGB_PATH.as_posix()}
depth_path: {DEPTH_PATH.as_posix()}
train: images/train
val: images/val
test: images/test
nc: 1
names: ['fresh_fruit_bunch']
"""
config_path.write_text(yaml_content)
print(f"Config: {config_path}")

In [ ]:
# Training config
CONFIG = {
    'model': 'yolo11n.pt',
    'epochs': 50,
    'batch': 16,
    'imgsz': 640,
    'device': 0 if torch.cuda.is_available() else 'cpu',
}
SEEDS = [42, 123]
EXP_PREFIX = 'exp_a4b_rgbd_synthetic'

print("⚠️ Note: 4-channel RGBD requires custom model modification")
print("Refer to train_a4b_rgbd_synthetic.py for implementation")

## Training Runs

**Implementation Note**: This requires modifying YOLOv11n first conv layer to accept 4 channels.
See `Experiments/scripts/train_a4b_rgbd_synthetic.py` for complete implementation.

In [ ]:
# Train both seeds
for seed in SEEDS:
    print(f"\n{'='*60}\nTRAINING SEED {seed}\n{'='*60}\n")
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    model = YOLO(CONFIG['model'])
    
    # TODO: Modify first conv layer for 4-channel input
    # See train_a4b_rgbd_synthetic.py for implementation
    
    results = model.train(
        data=str(config_path),
        epochs=CONFIG['epochs'],
        batch=CONFIG['batch'],
        imgsz=CONFIG['imgsz'],
        device=CONFIG['device'],
        name=f"{EXP_PREFIX}_seed{seed}",
        seed=seed,
        exist_ok=True
    )
    print(f"✅ Seed {seed} complete")

## Evaluation & Comparison

In [ ]:
# Evaluate
results_dict = {}
for seed in SEEDS:
    model_path = BASE_PATH / 'runs' / 'detect' / f"{EXP_PREFIX}_seed{seed}" / 'weights' / 'best.pt'
    model = YOLO(str(model_path))
    metrics = model.val(data=str(config_path), split='test', batch=CONFIG['batch'])
    
    results_dict[seed] = {
        'mAP50': metrics.box.map50,
        'mAP50-95': metrics.box.map,
        'precision': metrics.box.mp,
        'recall': metrics.box.mr
    }

# Summary
df = pd.DataFrame(results_dict).T
avg = df.mean()
avg.name = 'Average'
df = pd.concat([df, avg.to_frame().T])

print("\n" + "="*60)
print("A.4b RGB+SYNTHETIC DEPTH - RESULTS")
print("="*60 + "\n")
print(df.to_string(float_format=lambda x: f"{x:.3f}"))

# Multi-way comparison
print("\n" + "="*60)
print("COMPARISON: RGB vs REAL RGBD vs SYNTHETIC RGBD")
print("="*60 + "\n")
comparison = pd.DataFrame({
    'A.1 RGB': {'mAP50': 0.873, 'mAP50-95': 0.370},
    'A.3 Real RGBD': {'mAP50': 0.869, 'mAP50-95': 0.379},
    'A.4b Synthetic RGBD': {'mAP50': avg['mAP50'], 'mAP50-95': avg['mAP50-95']}
}).T
print(comparison.to_string(float_format=lambda x: f"{x:.3f}"))

# Analysis
gap_vs_rgb = avg['mAP50-95'] - 0.370
gap_vs_real_rgbd = avg['mAP50-95'] - 0.379
print(f"\nAnalysis:")
print(f"  Gap vs RGB (A.1): {gap_vs_rgb:+.3f}")
print(f"  Gap vs Real RGBD (A.3): {gap_vs_real_rgbd:+.3f}")
if gap_vs_real_rgbd > -0.01:
    print("  ✅ Synthetic RGBD matches Real RGBD! Depth sensor can be eliminated.")
elif gap_vs_rgb > 0:
    print("  ⚠️ Synthetic RGBD better than RGB but worse than Real RGBD")
else:
    print("  ❌ Synthetic depth not helpful for this task")

# Save
output_file = BASE_PATH / 'kaggleoutput' / 'test_rgbd_synthetic.txt'
output_file.parent.mkdir(exist_ok=True, parents=True)
with open(output_file, 'w') as f:
    f.write(f"A.4b RGB+Synthetic Depth Results\nGenerated: {datetime.now()}\n\n")
    f.write(df.to_string(float_format=lambda x: f"{x:.3f}"))
    f.write("\n\nComparison:\n")
    f.write(comparison.to_string(float_format=lambda x: f"{x:.3f}"))
print(f"\n✅ Saved: {output_file}")